# Introduction to Computation Graphs
- At the core of every TF program is the **computation graph**.
- A computation graph is a specific type of directed graph that is used for:
    - defining, unsurprisingly, computational structure
    
![](https://i.imgur.com/v9cLnlB.jpg)

The above example can be looked at as a simple equation:
$$f(1, 2) = 1 + 2 = 3$$

The fundamental building blocks of TF graphs are:
- **Nodes:**
    - Typically drawn as circles, ovals, or boxes
    - Represent some sort of computation or action being done on or with data in the graph's context.
    
- **Edges:**
    - actual values that get passed to and from Operations
    - typically drawn as arrows

# Defining Computation Graphs in TensorFlow
TensorFlow workflow pattern follows two steps:
- Define the computation graph
- Run the graph (with data)

# Building a TensorFlow Graph
![](https://i.imgur.com/ECJX8Iz.jpg)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [3]:
a = tf.constant(5, name="input_a")
b = tf.constant(3, name="input_b")
c = tf.multiply(a, b, name="mul_c")
d = tf.add(a, b, name="add_d")
e = tf.add(c, d, name="add_e")

In [4]:
sess = tf.Session()
sess.run(e)

23

In [5]:
sess.run(c)

15

In [6]:
output = sess.run(e)

In [11]:
# TensorBoard config
writer = tf.summary.FileWriter('./my_graph', sess.graph)

In [12]:
writer.close()
sess.close()

# Thinking with Tensors
- 1-D Tensor --> a vector
- 2-D Tensor --> a matrix

![](https://i.imgur.com/gzhEMCY.jpg)

This graph has several advantages over the previous example:
- The client only has to send input to a single node, which simplifies using the graph.
- The nodes that directly depend on the input now only have to keep track of one dependency instead of two.
- We now have the option of making the graph take in vectors of any length, if we'd like. This would make the graph more flexible.
- In TF, all the data passed from node to node are **Tensor** objects.
- Tensors are just a superset of matrices!

In [13]:
a = tf.constant([5, 3], name="input_a")
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")
d = tf.add(b, c, name="add_d")

In [14]:
sess = tf.Session()

In [15]:
sess.run(d)

23

# Python Native Types
- TF can take in Python numbers, booleans, strings, or lists of any of the above.
- Single values --> 0-D Tensor(or scalar)
- list of values --> 1-D Tensor(vector)
- list of lists of values -> 2-D Tensor(matrix)

# NumPy arrays
- TF is tightly integrated with **NumPy**
- TF data types are based on those from NumPy

In [16]:
np.int32 == tf.int32

True

- Any NumPy array can be passed into any TF Op.

# TensorFlow operations
- Are nodes that perform computations on or with Tensor objects
- After computation, they return zero or more tensors.

## Overloaded operators
- TF also overloads common mathematical operators to make multiplication, addition, subtraction, and other common operations more concise.



# TensorFlow graphs
 Creating a Graph is simple-its constructor doesn't take any variables:

In [17]:
# create a new graph
g = tf.Graph()

Once we have our Graph initialized, we can add Operations to it by using the ```Graph.as_default()``` method to access its context manager.

In [18]:
with g.as_default():
    # create Operations as usual; they will be adding to graph 'g'
    a = tf.multiply(2, 3)

- TF automatically creates a ```Graph``` when the library is loaded and assigns it to be the default. Thus, any Operations, tensors, etc. defined outside of a ```Graph.as_default()``` context manager will automatically be placed in the default graph:

In [19]:
# Placed in the default graph
in_default_graph = tf.add(1, 2)

In [20]:
# Placed in graph 'g'
with g.as_default():
    in_graph_g = tf.multiply(2, 3)

In [21]:
# We are no longer in the with block, so this is placed in the default graph
also_in_default_graph = tf.subtract(5, 1)

In [22]:
# To get a handle to the default graph, use the ```tf.get_default_graph()``` function:
default_graph = tf.get_default_graph()

# TensorFlow Sessions
- Are responsible for graph execution.
- The constructor takes in three optional parameters:
    - ```target``` specifies the execution engine to use
    - ```graph``` specifies the ```Graph``` object that will be launched in the ```session```. The default value is ```None```.
    - ```config``` allows users to specify options to configure the session, such as limiting the number of CPUs or GPUs to use, setting optimization parameters for graphs, and logging options.

In [23]:
a = tf.add(2, 5)
b = tf.multiply(a, 3)

# Start up a session using the default graph
sess = tf.Session()

Once a **Session** is opened, you can use its primary method, ```run()```, to calculate the value of a desired ```Tensor``` output

In [24]:
sess.run(b)

21

```Session.run()``` takes in one required parameter, **```fetches```**, as well as three optional parameters:
- ```feed_dict```
- ```options```
- ```run_metadata```

## 1. Fetches
- ```fetches``` accepts any graph element (either an operation or Tensor object), which specifies what the user would like to execute.
- If the requested object is a **Tensor**, then the output of ```run()``` will be a NumPy array.
- If the object is an ```Operation```, then the output will be ```None```.

In [25]:
sess.run(b) # we set fetches to the tensor b

21

In [26]:
sess.run([a, b]) # pass in a list of graph elements

[7, 21]

- When ```fetches``` is a list, the output of ```run()``` will be a list with values corresponding to the output of the requested elements.
- In addition using ```fetches``` to get ```Tensor``` outputs, we can give ```fetches``` a direct handle to an **operation** whcih is a useful side-effect when run.
    - Example: ```tf.global_variables_initializer()```

In [28]:
# Performs the computations needed to initialize Variables, but returns ```None```
sess.run(tf.global_variables_initializer())

## 2. Feed Dictionary
- The parameter ```feed_dict``` is used to override ```Tensor``` values in the graph, and it expects a Python dictionary object as input.
- The keys in the dictionary are handles to ```Tensor``` objects that should be overridden, while the values can be numbers, strings, lists, or NumPy arrays.
- The values must of the same type.

In [29]:
# create Operations, Tensors, etc (using teh default graph)
a = tf.add(2, 5)
b = tf.multiply(a, 3)

In [30]:
# Start up a Session using the default graph
sess = tf.Session()

In [31]:
# Define a dictionary that says to replace the value of a with 15
replace_dict = {a:15}


In [32]:
# Run the session, passing in replace_dict as the value to feed_dict
sess.run(b, feed_dict=replace_dict)

45

In [33]:
sess.close()